#Instructions

add segmented routes and OSM Highway Export to map

segmented routes: https://azgeo.maps.arcgis.com/home/item.html?id=b82a0623026c42ea96bdc12dc42967ea

OpenStreetMap Highways for North America:
https://www.arcgis.com/home/item.html?id=7afec250e02845868db89c83949a672f


To improve reliability, rather than join based on centroid, I should generate three or five points along each line and then use a voting method...

In [ ]:
def select_pattern(feature_layer, field, pattern, select_type = "NEW SELECTION"):
    matcher = re.compile(pattern)
    describe_feature = arcpy.Describe(feature_layer)
    
    oid_list = []
    with arcpy.da.SearchCursor(feature_layer, ['OID@', field]) as sc:
        for row in sc:
            if matcher.search(row[1]) is not None:
                oid_list.append(row[0])
    if(len(oid_list)) == 0:
        print('No records matched.')
        return None
    
    sql = "\"{}\" IN ({})".format(
        describe_feature.OIDFieldName,
        ",".join(["{}".format(oid) for oid in oid_list])
    )

    arcpy.SelectLayerByAttribute_management(
        feature_layer,
        select_type,
        sql
    )

In [194]:
pattern = '\s{1,4}0[\s123]$' # Match 0 at end
select_pattern("segmented_routes", 'route_id', pattern)

In [189]:
arcpy.management.SelectLayerByAttribute("segmented_routes", "REMOVE_FROM_SELECTION", """type = 'dual_line'""", None)

<Result 'segmented_routes'>

### resume here ... I need to separate into three layers

* 1 line to 1 line
* 2 lines to 2 lines
* 2 lines to 1 line
* possibly ... 2 lines to 2 lines

In [ ]:
arcpy.management.CalculateField(
    "segmented_routes", "type", "'dual_line'", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.management.SelectLayerByAttribute(
    "segmented_routes", "SWITCH_SELECTION", "functional_class IN (1, 2)", None)

arcpy.management.CalculateField(
    "segmented_routes", "type", "'single_line'", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")


In [ ]:
# separate motorways from facilities represnted by one line from facilities represented by two lines
# ... should actually just separate by one way and two way, then separate the centroids by one way and two two
arcpy.management.SelectLayerByAttribute(
    "OSM_NA_Highway_Export", "NEW_SELECTION", "highway = 'motorway'", None)
arcpy.conversion.ExportFeatures(
    "OSM_NA_Highway_Export", r"Y:\mag\sun_cloud\layers\justice_40_users\j40_users.gdb\OSM_Non_Motorways", '', "NOT_USE_ALIAS", 'highway "highway" true true false 255 Text 0 0,First,#,OSM_NA_Highway_Export,highway,0,255;name "name" true true false 255 Text 0 0,First,#,OSM_NA_Highway_Export,name,0,255;oneway "oneway" true true false 255 Text 0 0,First,#,OSM_NA_Highway_Export,oneway,0,255;osm_id2 "osm_id2" true true false 255 Text 0 0,First,#,OSM_NA_Highway_Export,osm_id2,0,255;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,OSM_NA_Highway_Export,Shape_Length,-1,-1', None)
arcpy.management.SelectLayerByAttribute(
    "OSM_NA_Highway_Export", "NEW_SELECTION", "highway <> 'motorway'", None)
arcpy.conversion.ExportFeatures(
    "OSM_NA_Highway_Export", r"Y:\mag\sun_cloud\layers\justice_40_users\j40_users.gdb\OSM_Motorways", '', "NOT_USE_ALIAS", 'highway "highway" true true false 255 Text 0 0,First,#,OSM_NA_Highway_Export,highway,0,255;name "name" true true false 255 Text 0 0,First,#,OSM_NA_Highway_Export,name,0,255;oneway "oneway" true true false 255 Text 0 0,First,#,OSM_NA_Highway_Export,oneway,0,255;osm_id2 "osm_id2" true true false 255 Text 0 0,First,#,OSM_NA_Highway_Export,osm_id2,0,255;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,OSM_NA_Highway_Export,Shape_Length,-1,-1', None)

In [ ]:
arcpy.management.SelectLayerByAttribute(
    "OSM_Non_Motorways", "NEW_SELECTION", "oneway IN ('yes', '-1')", None)
arcpy.conversion.ExportFeatures(
    "OSM_Non_Motorways", r"j40_users.gdb\OSM_Non_Motorway_Two_Lines"
)

In [ ]:
arcpy.analysis.PairwiseBuffer(
    "segmented_routes", r"Y:\mag\sun_cloud\layers\justice_40_users\j40_users.gdb\segmented_rout_PairwiseBuffe", "8 Meters", "ALL", None, "PLANAR", "0 Feet")
arcpy.management.SelectLayerByLocation(
    "OSM_Non_Motorways", "WITHIN", "segmented_rout_PairwiseBuffe", None, "NEW_SELECTION", "NOT_INVERT")
arcpy.management.SelectLayerByAttribute(
    "OSM_Non_Motorways", "REMOVE_FROM_SELECTION", "oneway IN ('yes', '-1')", None)
arcpy.conversion.ExportFeatures(
    "OSM_Non_Motorways", r"Y:\mag\sun_cloud\layers\justice_40_users\j40_users.gdb\OSM_Non_Motorway_One_Line_Filtered", '', "NOT_USE_ALIAS", 'highway "highway" true true false 255 Text 0 0,First,#,OSM_Non_Motorways,highway,0,255;name "name" true true false 255 Text 0 0,First,#,OSM_Non_Motorways,name,0,255;oneway "oneway" true true false 255 Text 0 0,First,#,OSM_Non_Motorways,oneway,0,255;osm_id2 "osm_id2" true true false 255 Text 0 0,First,#,OSM_Non_Motorways,osm_id2,0,255;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,OSM_Non_Motorways,Shape_Length,-1,-1', None)


In [120]:
# manually go through and delete SC segments
# that do not overlap with OSM segments
# ... should consider removing these
# segments from the SC Routes layer

# then go through and filter the OSM segments down to just where
# they overlap with SC routes to get rid of the lower
# funtional class routes

In [198]:
# create centroids
arcpy.management.FeatureToPoint(
    "segmented_routes", 
    r"j40_users.gdb\centroids", 
    "INSIDE"
)

<Result 'j40_users.gdb\\centroids'>

In [204]:
# select single vs dual and buffer
arcpy.management.SelectLayerByAttribute(
    "centroids", 
    "NEW_SELECTION", 
    "type = 'dual_line'", 
    None
)

<Result 'centroids'>

In [ ]:
arcpy.conversion.ExportFeatures(
    "centroids", 
    r"j40_users.gdb\centroids_dual_line", 
    '', "NOT_USE_ALIAS"
)

In [ ]:
arcpy.edit.Snap(
    "centroids_dual_line", 
    "OSM_Dual_Line_Features EDGE '20 Meters'"
)

In [ ]:
arcpy.analysis.PairwiseIntersect(
    "centroids_dual_line;OSM_Dual_Line_Features", 
    r"j40_users.gdb\centroids_dual_inter", 
    "ALL", None, "INPUT"
)

In [209]:
# export out the single-line centroids
# to join to single line and dual line OSM
arcpy.management.SelectLayerByAttribute(
    "centroids", 
    "NEW_SELECTION", 
    "type = 'single_line'", 
    None
)
arcpy.analysis.PairwiseBuffer(
    "centroids", 
    r"j40_users.gdb\centroids_single_line_buffer", 
    "15 Meters", "NONE", None, "PLANAR", "0 Feet"
)

<Result 'j40_users.gdb\\centroids_single_line_buffer'>

In [ ]:
arcpy.management.AddSpatialJoin(
    "centroids_single_line_buffer", 
    "OSM_Single_Line_Features", 
    "JOIN_ONE_TO_ONE", 
    "KEEP_ALL", r'highway "highway" true true false 255 Text 0 0,First,#,Y:\mag\sun_cloud\layers\justice_40_users\j40_users.gdb\OSM_Non_Motorway_One_L_Merge,highway,0,255;name "name" true true false 255 Text 0 0,First,#,Y:\mag\sun_cloud\layers\justice_40_users\j40_users.gdb\OSM_Non_Motorway_One_L_Merge,name,0,255;oneway "oneway" true true false 255 Text 0 0,First,#,Y:\mag\sun_cloud\layers\justice_40_users\j40_users.gdb\OSM_Non_Motorway_One_L_Merge,oneway,0,255;osm_id2 "osm_id2" true true false 255 Text 0 0,First,#,Y:\mag\sun_cloud\layers\justice_40_users\j40_users.gdb\OSM_Non_Motorway_One_L_Merge,osm_id2,0,255;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,Y:\mag\sun_cloud\layers\justice_40_users\j40_users.gdb\OSM_Non_Motorway_One_L_Merge,Shape_Length,-1,-1', "LARGEST_OVERLAP", None, '')

In [ ]:
arcpy.management.SelectLayerByAttribute(
    "centroids_single_line_buffer", 
    "NEW_SELECTION", 
    "centroids_single_line_buffer_AddSpatialJoin.osm_id2 IS NULL", 
    None
)

In [210]:
# filter OSM routes down to just the layers of interest

In [ ]:
arcpy.management.SelectLayerByAttribute(
    "OSM_Dual_Line_Features", 
    "NEW_SELECTION", 
    "highway IN ('trunk', 'motorway')", 
    "INVERT"
)
arcpy.conversion.ExportFeatures(
    "OSM_Dual_Line_Features", 
    r"j40_users.gdb\OSM_Dual_Line_No_Motorways", 
    '', "NOT_USE_ALIAS"
)

In [ ]:
arcpy.analysis.PairwiseIntersect(
    "centroids_one_to_two_buff20;OSM_Dual_Line_No_Motorways", 
    r"j40_users.gdb\centroids_one_to_two_intersect", 
    "ALL", None, "INPUT"
)

In [ ]:
# export things out
arcpy.conversion.ExportTable(
    "centroids_one_to_two_intersect", 
    r"centroids_one_to_two_intersect.csv",    
)

# Swith to R and BigQuery

now process things in R becuase my python skillz are lacking

1. Run 1_extract_osm_ids.R
2. Run both bigquery queries
3. Then run 2_combine.R and come back here

In [3]:
arcpy.management.AddJoin(
    "segmented_routes", 
    "join_id", "trips_by_join_id.csv", 
    "join_id", "KEEP_COMMON", "NO_INDEX_JOIN_FIELDS")
arcpy.conversion.ExportFeatures(
    "segmented_routes", 
    r"j40_users.gdb\disadvantaged_facility_users"
)

<Result 'Y:\\mag\\sun_cloud\\layers\\justice_40_users\\j40_users.gdb\\disadvantaged_facility_users'>

In [ ]:
# looking pretty good
# definitely some gaps - but these seem to be on the 
# OSM / Replica side. Could improve by joining based on multiple points per segment


In [4]:
working_layer = 'disadvantaged_facility_users'

system_fields = [arcpy.Describe(working_layer).OIDFieldName, 'Shape', 'geom', 'Shape_Length']
current_fields = [f.name for f in arcpy.ListFields(working_layer) if not f.name in system_fields]

# Define tuples of CurrentFieldName, target_field_name, Alias
target_fields = [
    ('route_id', 'route_id', 'Route ID'),
    ('trips', 'trips', 'Total Person Trips (Replica Estimate)'),
    ('j40_trips', 'j40_trips', 'Trips by Disadvantaged Users'),
    ('percent_j40', 'percent_j40', 'Percent Disadvantaged Users'),
]

# delete unused fields
delete_fields = [field for field in current_fields if not field in [a for a, b, c in target_fields]]
if delete_fields:
    arcpy.management.DeleteField(working_layer, delete_fields)
    
for current, new, alias in target_fields:
    # arcgis can't change capitalization ... a hacky workaround
    if new != current and new.lower() == current.lower():
        arcpy.management.AlterField(working_layer, current, current + "1")
        current = current + "1"
    arcpy.management.AlterField(working_layer, current, new, alias)